# Merge `drug price data` with `patent_data` for control group

In [94]:
# import module
import numpy as np
import pandas as pd

In [95]:
# read patent data with drugs that expired in 2019
patent_2019 = pd.read_excel('2017_22.xlsx')
patent_2019.head(3)

,INGREDIENT_STRING,DOSAGE_FORM,ROUTE,TRADE_NAME,APPL_TYPE,APPL_NO,PRODUCT_NO,PATENT_NO,EXPIRE_DATE,PE_EXTENSION_FLAG,PE_EXTENSION_DATE,DRUG_SUBSTANCE_FLAG,DRUG_PRODUCT_FLAG,USE_CODE,DELIST_FLAG,SUBMISSION_DATE
0,PEGAPTANIB SODIUM,INJECTABLE,INTRAVITREAL,MACUGEN,N,21756,1,6011020,2017-01-04,N,01/04/2017,Y,N,NaN,N,NaN
1,EXENATIDE SYNTHETIC,"FOR SUSPENSION, EXTENDED RELEASE",SUBCUTANEOUS,BYDUREON,N,22200,1,6858576,2017-01-06,N,01/06/2017,N,N,U-656,N,NaN
2,EXENATIDE SYNTHETIC,"FOR SUSPENSION, EXTENDED RELEASE",SUBCUTANEOUS,BYDUREON PEN,N,22200,2,6858576,2017-01-06,N,01/06/2017,N,N,U-656,N,"Mar 28, 2014"


In [96]:
# get a list of unique patent names
unique_names  = patent_2019.TRADE_NAME.unique()

In [97]:
# read drug price data
df = pd.read_csv('drug_data.csv')
df.head(3)

,Brnd_Name,Gnrc_Name,Tot_Mftr,Mftr_Name,Tot_Spndng_2017,Tot_Dsg_Unts_2017,Tot_Clms_2017,Tot_Benes_2017,Avg_Spnd_Per_Dsg_Unt_Wghtd_2017,Avg_Spnd_Per_Clm_2017,...,Tot_Spndng_2021,Tot_Dsg_Unts_2021,Tot_Clms_2021,Tot_Benes_2021,Avg_Spnd_Per_Dsg_Unt_Wghtd_2021,Avg_Spnd_Per_Clm_2021,Avg_Spnd_Per_Bene_2021,Outlier_Flag_2021,Chg_Avg_Spnd_Per_Dsg_Unt_20_21,CAGR_Avg_Spnd_Per_Dsg_Unt_17_21
0,1ST TIER UNIFINE PENTIPS,"Pen Needle, Diabetic",1,Overall,217938.04,1014318.0,8595.0,3025.0,0.216105,25.356375,...,102280.76,459384.0,3708,1313.0,0.223001,27.583808,77.898522,0,0.024346,0.007885
1,1ST TIER UNIFINE PENTIPS,"Pen Needle, Diabetic",1,Owen Mumford Us,217938.04,1014318.0,8595.0,3025.0,0.216105,25.356375,...,102280.76,459384.0,3708,1313.0,0.223001,27.583808,77.898522,0,0.024346,0.007885
2,1ST TIER UNIFINE PENTIPS PLUS,"Pen Needle, Diabetic",1,Overall,402124.68,1850703.0,15403.0,5967.0,0.217359,26.106906,...,131927.33,566872.0,4564,1766.0,0.232812,28.906076,74.704037,0,0.159125,0.017317


In [98]:
# turn both generic name in the price data and the unique patent name all into upper class for merging
df1 = df.copy()
unique_names = [i.upper() for i in unique_names] # unique patent expired in 2019
df1["Brnd_Name"] = df1["Brnd_Name"].str.upper()


In [99]:
# merge on 'Brnd Name' and 'Trade Name'
merge_name = []
for name in df1["Brnd_Name"]:
    for unique_patent in unique_names:
        if (name in unique_patent) or (unique_patent in name):
            merge_name.append(name)
            break
        else:
            pass


In [100]:
merge_name = set(merge_name)
merge_df = df1[df1['Brnd_Name'].isin(merge_name)]
merge_df
df4 = df.copy()
df4_merged = df4.merge(merge_df, indicator=True, how='outer')
result_df4 = df4_merged.query('_merge=="left_only"').drop('_merge', axis=1)
result_df4 = result_df4.reset_index(drop=True)
merge_df = result_df4
merge_df

,Brnd_Name,Gnrc_Name,Tot_Mftr,Mftr_Name,Tot_Spndng_2017,Tot_Dsg_Unts_2017,Tot_Clms_2017,Tot_Benes_2017,Avg_Spnd_Per_Dsg_Unt_Wghtd_2017,Avg_Spnd_Per_Clm_2017,...,Tot_Spndng_2021,Tot_Dsg_Unts_2021,Tot_Clms_2021,Tot_Benes_2021,Avg_Spnd_Per_Dsg_Unt_Wghtd_2021,Avg_Spnd_Per_Clm_2021,Avg_Spnd_Per_Bene_2021,Outlier_Flag_2021,Chg_Avg_Spnd_Per_Dsg_Unt_20_21,CAGR_Avg_Spnd_Per_Dsg_Unt_17_21
0,1ST TIER UNIFINE PENTIPS,"Pen Needle, Diabetic",1,Overall,2.179380e+05,1014318.0,8595.0,3025.0,0.216105,25.356375,...,1.022808e+05,459384.0,3708,1313.0,0.223001,27.583808,77.898522,0,0.024346,0.007885
1,1ST TIER UNIFINE PENTIPS,"Pen Needle, Diabetic",1,Owen Mumford Us,2.179380e+05,1014318.0,8595.0,3025.0,0.216105,25.356375,...,1.022808e+05,459384.0,3708,1313.0,0.223001,27.583808,77.898522,0,0.024346,0.007885
2,1ST TIER UNIFINE PENTIPS PLUS,"Pen Needle, Diabetic",1,Overall,4.021247e+05,1850703.0,15403.0,5967.0,0.217359,26.106906,...,1.319273e+05,566872.0,4564,1766.0,0.232812,28.906076,74.704037,0,0.159125,0.017317
3,1ST TIER UNIFINE PENTIPS PLUS,"Pen Needle, Diabetic",1,Owen Mumford Us,4.021247e+05,1850703.0,15403.0,5967.0,0.217359,26.106906,...,1.319273e+05,566872.0,4564,1766.0,0.232812,28.906076,74.704037,0,0.159125,0.017317
4,ABACAVIR,Abacavir Sulfate,7,Overall,1.338625e+07,3189707.0,48283.0,8090.0,4.298589,277.245616,...,7.038594e+06,2501447.0,30540,4254.0,3.187698,230.471311,1654.582471,0,-0.087066,-0.072022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12407,ZYPITAMAG,Pitavastatin Magnesium,1,Overall,NaN,NaN,NaN,NaN,NaN,NaN,...,4.213602e+05,53771.0,1027,418.0,7.836376,410.282600,1008.038828,0,0.011418,0.006163
12408,ZYPITAMAG,Pitavastatin Magnesium,1,Medicure Intern,NaN,NaN,NaN,NaN,NaN,NaN,...,4.213602e+05,53771.0,1027,418.0,7.836376,410.282600,1008.038828,0,0.011418,0.006163
12409,ZYTIGA,Abiraterone Acetate,2,Overall,9.352092e+08,11471704.0,99705.0,19098.0,83.879924,9379.762006,...,1.278258e+08,903474.0,12303,2329.0,155.611256,10389.806210,54884.407820,0,-0.054350,0.167067
12410,ZYTIGA,Abiraterone Acetate,1,Janssen Pharm.,5.800776e+07,363568.0,6076.0,2142.0,159.551338,9547.031093,...,9.785922e+07,538888.0,9087,1752.0,181.594724,10769.144700,55855.717950,0,-0.012171,0.032882


In [101]:
# drop drug with multiple manufactuere, i.e. drug with "Mftr == Overall && "Tot_Mftr >= 2""
drop_names = merge_df[(merge_df.Mftr_Name == 'Overall') & (merge_df.Tot_Mftr >=2)].Brnd_Name.unique()
merge_df = merge_df[merge_df.Brnd_Name.isin(drop_names) == False]
merge_df.shape

(3796, 46)

In [102]:
# drop 'Mftr_Name' == "Overall" since it's an average of all that generic drug
merge_df2 = merge_df[merge_df.Mftr_Name != 'Overall']


In [103]:
# check how many unic drug
merge_df2.Brnd_Name.unique().shape

(1828,)

In [104]:
# check for number of unique drug after dropping missing values 
merge_df2.dropna().Brnd_Name.unique().shape

(1106,)

In [105]:
merge_df3 = merge_df2.dropna()


In [106]:
merge_df3.reset_index(inplace = True, drop = True)

In [107]:
merge_df3.to_csv('control_price_patent.csv')

In [108]:
merge_df3

,Brnd_Name,Gnrc_Name,Tot_Mftr,Mftr_Name,Tot_Spndng_2017,Tot_Dsg_Unts_2017,Tot_Clms_2017,Tot_Benes_2017,Avg_Spnd_Per_Dsg_Unt_Wghtd_2017,Avg_Spnd_Per_Clm_2017,...,Tot_Spndng_2021,Tot_Dsg_Unts_2021,Tot_Clms_2021,Tot_Benes_2021,Avg_Spnd_Per_Dsg_Unt_Wghtd_2021,Avg_Spnd_Per_Clm_2021,Avg_Spnd_Per_Bene_2021,Outlier_Flag_2021,Chg_Avg_Spnd_Per_Dsg_Unt_20_21,CAGR_Avg_Spnd_Per_Dsg_Unt_17_21
0,1ST TIER UNIFINE PENTIPS,"Pen Needle, Diabetic",1,Owen Mumford Us,217938.04,1014318.0,8595.0,3025.0,0.216105,25.356375,...,102280.76,459384.00,3708,1313.0,0.223001,27.583808,77.898522,0,0.024346,0.007885
1,1ST TIER UNIFINE PENTIPS PLUS,"Pen Needle, Diabetic",1,Owen Mumford Us,402124.68,1850703.0,15403.0,5967.0,0.217359,26.106906,...,131927.33,566872.00,4564,1766.0,0.232812,28.906076,74.704037,0,0.159125,0.017317
2,ABACAVIR-LAMIVUDINE-ZIDOVUDINE,Abacavir/Lamivudine/Zidovudine,1,Lupin Pharmaceu,10158493.71,468341.0,7451.0,971.0,21.690379,1363.373200,...,146795.20,7002.00,118,32.0,20.964753,1244.027119,4587.350000,0,-0.031556,-0.008470
3,ACCOLATE,Zafirlukast,1,Par Pharmaceuti,374889.01,100896.0,1082.0,190.0,3.715296,346.477828,...,217404.57,57295.00,521,103.0,3.794640,417.283244,2110.723981,0,0.019205,0.005297
4,ACCUPRIL,Quinapril HCl,1,Pfizer US Pharm,2577686.04,633391.0,8084.0,1728.0,4.068535,318.862697,...,1802400.49,386511.00,4302,1045.0,4.665907,418.968036,1724.785158,0,0.016517,0.034843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1143,ZOVIRAX*,Acyclovir,1,Bausch Health U,15861318.10,127606.0,16337.0,11315.0,140.664500,970.883155,...,584006.17,14440.00,900,462.0,77.373925,648.895744,1264.082619,0,-0.439464,-0.138803
1144,ZUPLENZ,Ondansetron,1,Midatech Pharma,114061.49,3971.0,86.0,62.0,28.723732,1326.296395,...,140480.70,3803.00,83,57.0,36.940251,1692.538554,2464.573684,0,0.033366,0.064915
1145,ZYCLARA,Imiquimod,1,Bausch Health U,1950803.57,21632.0,1608.0,1153.0,120.629172,1213.186300,...,141751.69,1284.50,107,76.0,140.833506,1324.782150,1865.153816,0,0.333520,0.039473
1146,ZYFLO,Zileuton,1,"Chiesi USA, Inc",3878417.60,129845.0,1136.0,200.0,29.869595,3414.100000,...,3649394.42,115468.00,870,163.0,31.605245,4194.706230,22388.922820,0,0.013384,0.014221
